In [1]:
# imports
%matplotlib inline
import re
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import glob

In [25]:
mu = 1.5e-8  # mutation rate
r = 1.0e-8  # recombination rate
N = 10000  # diploid population size in the sexual ancestor
chromLength = int(1e7)  # 10 Mbp per chromosome
numChromosomes = 100  # simulate 100 chromosomes per replicate

In [26]:
# scaled parameters for entire chromosomes
rho = 4*N*r*chromLength
theta = 4*N*mu*chromLength

In [27]:
numReps = 5

In [28]:
# the different transition times to test
asexGens = np.array([2000, 5000, 10000, 20000, 40000])
asexGensRescaled = asexGens / (2.0*N)  # rescaled by 2*N

In [29]:
# generate ms commands (but using scrm instead of ms)
# scrm: http://scrm.github.io/ 

msCmds = open('sim_ms_commands.txt', 'w')
for asexGen in asexGensRescaled:
    for rep in xrange(1, numReps+1):
        filename = "sim_data_t{}_rho{}_asex{}_rep{}".format(theta, rho, asexGen, rep)
        msCmd = "scrm 2 {numChroms} -t {theta} -r {rho} {chromLen} | ./add_asex.py - {asexGen} > {filename}".format(numChroms = numChromosomes,
                                                                   theta = theta, rho = rho,
                                                                   chromLen = chromLength, asexGen = asexGen,
                                                                   filename = filename)
        msCmds.write(msCmd+'\n')
# control runs with constant size and no history of asex
for rep in xrange(1,numReps+1):
    filename = "sim_data_t{}_rho{}_no_asex_rep{}".format(theta, rho, rep)
    msCmd = "scrm 2 {numChroms} -t {theta} -r {rho} {chromLen} > {filename}".format(numChroms = numChromosomes,
                                                               theta = theta, rho = rho,
                                                               chromLen = chromLength,
                                                               filename = filename)
    msCmds.write(msCmd+'\n')
msCmds.close()

Simulate the data.

    cat sim_ms_commands.txt | parallel -n 4
    mkdir -p simdata
    mv sim_data* simdata/

Convert to psmc format.
    
    for file in simdata/sim_data_*
    do
        psmcfilename="$file.psmc"
        ./ms2psmcfa.py $file > $psmcfilename
    done

In [ ]:
datadir = "simdata"

In [69]:
# make psmc commands

initialT = "0.5"

psmcCmds = open('sim_psmc_commands.txt', 'w')

for asexGen in asexGensRescaled:
    for rep in xrange(1, numReps+1):
        datafilename = "{}/sim_data_t{}_rho{}_asex{}_rep{}.psmc".format(datadir, theta, rho, asexGen, rep)
        resultsfilename = "{}/results_data_t{}_rho{}_asex{}_rep{}".format(datadir, theta, rho, asexGen, rep)
        resultsfilenameT = "{}/results_data_t{}_rho{}_asex{}_rep{}_T".format(datadir, theta, rho, asexGen, rep)
        psmcCmd = "psmc {} > {}".format(datafilename, resultsfilename)
        psmcCmdT = "psmc {} -T {} > {}".format(datafilename, initialT, resultsfilenameT)
        psmcCmds.write(psmcCmd+'\n')
        psmcCmds.write(psmcCmdT+'\n')
# control runs with constant size and no history of asex
for rep in xrange(1,numReps+1):
    datafilename = "{}/sim_data_t{}_rho{}_no_asex_rep{}.psmc".format(datadir, theta, rho, asexGen, rep)
    resultsfilename = "{}/results_data_t{}_rho{}_no_asex_rep{}".format(datadir, theta, rho, asexGen, rep)
    resultsfilenameT = "{}/results_data_t{}_rho{}_no_asex_rep{}_T".format(datadir, theta, rho, asexGen, rep)
    psmcCmd = "psmc {} > {}".format(datafilename, resultsfilename)
    psmcCmdT = "psmc {} -T {} > {}".format(datafilename, initialT, resultsfilenameT)
    psmcCmds.write(psmcCmd+'\n')
    psmcCmds.write(psmcCmdT+'\n')
psmcCmds.close()

In [48]:
# read and return psmc results as dict
def read_psmc_results(filename):
    results = {}
    results['divtime'] = None  # divtime None by default...
    fin = open(filename,'r')
    for line in fin:
        line = line.strip()
        if line[:2] == 'TR':
            spline = line.split('\t')
            results['theta'] = float(spline[1])
            results['rho'] = float(spline[2])
            continue
        if line[:2] == 'DT':
            results['divtime'] = float(line.split('\t')[1])
            continue
        if line[:2] == 'RS':
            spline = line.split('\t')
            i = int(spline[1])
            ti = float(spline[2])
            lambdai = float(spline[3])
            results['maxi'] = i
            results['t{}'.format(i)] = ti
            results['lambda{}'.format(i)] = lambdai
            continue
    return results

In [82]:
rows = {}
for asexGen in asexGensRescaled:
    for rep in xrange(1, numReps+1):
        index = 'asex{}_rep{}'.format(asexGen, rep)
        resultsfilename = "{}/results_data_t{}_rho{}_asex{}_rep{}".format(datadir, theta, rho, asexGen, rep)
        try:
            rows[index] = read_psmc_results(resultsfilename)
        except:
            pass
        index = 'asex{}_rep{}_T'.format(asexGen, rep)
        resultsfilename = "{}/results_data_t{}_rho{}_asex{}_rep{}_T".format(datadir, theta, rho, asexGen, rep)
        try:
            rows[index] = read_psmc_results(resultsfilename)
        except:
            pass
for rep in xrange(1, numReps+1):
    resultsfilename = "{}/results_data_t{}_rho{}_no_asex_rep{}".format(datadir, theta, rho, asexGen, rep)
    

In [83]:
dat = pd.DataFrame.from_dict(rows, orient = 'index')
maxi = dat['maxi'].max()
newCols = ['theta', 'rho', 'divtime']
newCols.extend(['lambda'+str(i) for i in xrange(0,maxi+1)])
newCols.extend(['t'+str(i) for i in xrange(0,maxi+1)])
newCols.append('maxi')
dat = dat[newCols]

In [84]:
# rescaling the population size

binSize = 100  # we binned the bp's by 100 in the input to PSMC

dat['lambda0'] = dat['theta'] / (4.0*mu) / binSize

for i in xrange(1, maxi+1):
    colLam = 'lambda'+str(i)
    colT = 't'+str(i)
    dat[colLam] = dat['lambda0'] * dat[colLam]
    dat[colT] = 2.0*dat['lambda0'] * dat[colT]